Saleh Ahmad
i200605@nu.edu.pk

In [9]:
import numpy as np
from collections import defaultdict
import random

In [10]:
class Laplace_LM_Roman_Urdu():
    def __init__(self,ngram_type=None):
        if ngram_type == None:
            self.ngram = 2
        else:
            self.ngram = ngram_type

    def fit(self,TrainCorpusVocab): #TrainCorpus = all tokens in order of occurence
        Vocabs = defaultdict(int)
        UniqueVocabSize = defaultdict(int)
        VocabSize = defaultdict(int)
        for ngram in range(self.ngram,0,-1):
            IdxToken = 0
            ngrams = defaultdict(int)
            Len_Vocab = len(TrainCorpusVocab)
            while IdxToken < Len_Vocab-ngram+1:
                current_n_gram = TrainCorpusVocab[IdxToken]
                for idx in range(1,ngram):
                    current_n_gram += ' ' + TrainCorpusVocab[IdxToken+idx]
                
                ngrams[current_n_gram] += 1
                IdxToken += 1

            ngrams = {k:v for k,v in ngrams.items() if v > 0}
            Vocabs[ngram] = ngrams
            UniqueVocabSize[ngram] = len(ngrams)
            VocabSize[ngram] = sum(ngrams.values())
        
        self.Vocabs = Vocabs
        self.UniqueVocabSizes = UniqueVocabSize
        self.VocabSizes = VocabSize

    def __Get_Random_Length_Of_Verse(self):
        #generate random length b/w 5 and 9
        return random.randint(6,8)

    def __Get_Random_Starting_Token(self,s_tokens):
        #generate random starting token from s_tokens which is a list of starting tokens
        return random.choice(s_tokens)
    
    def __Print_Vocab(self):
        print(self.Vocabs)
    
    def __Get_Next_Word_From_Candidate(self,Sequence):
        ngram_ToUse = len(Sequence.split()) + 1
        Candidates = [ngram for ngram in self.Vocabs[ngram_ToUse].keys() if ' '.join(ngram.split()[0:-1]) == Sequence]
        if len(Candidates) == 0:
            return self.__Get_Next_Word_From_Candidate(' '.join(Sequence.split()[1:])) #BackOff
        else:
            Probs_Of_Candidates = np.zeros_like(Candidates)
            for idxCandidate,candidate in enumerate(Candidates):
                Probs_Of_Candidates[idxCandidate] = (self.Vocabs[ngram_ToUse][candidate] + 1) / (self.VocabSizes[self.ngram] + self.UniqueVocabSizes[self.ngram])
                
            return Candidates[np.argmax(Probs_Of_Candidates)].split()[-1] #return max prob and the next word

    def __ReverseBackOff(self,sequence): 
        '''Initially the length of the sequence will be one. It will try to complete the sequence to the self.ngrams-1 length mark.
        '''
        for ngramIdx in range(2,self.ngram):
            Candidates = [ngram for ngram in self.Vocabs[ngramIdx].keys() if ' '.join(ngram.split()[0:ngramIdx-1]) == sequence]
            Probs_Of_Candidates = np.zeros_like(Candidates)
            for idxCandidate,candidate in enumerate(Candidates):
                Probs_Of_Candidates = (self.Vocabs[ngramIdx][candidate] + 1) / (self.VocabSizes[self.ngram] + self.UniqueVocabSizes[self.ngram])
            Seq = sequence + ' ' + str(Candidates[np.argmax(Probs_Of_Candidates)].split()[-1])
            sequence = Seq
        return sequence


    def predict(self,StartingTokens): #StartingTokens = list of repetitive tokens, can extract unique and count
        '''
        Prediction format
        1 stanza = 14 verses (1 * 14)
        1 verse = 6 - 8 words
        '''         

        Verse = ''
        for idxVerse in range(0,14):
            StartingTokenOfThisVerse = self.__Get_Random_Starting_Token(StartingTokens) #Verse starts with a random starting token, length is 1 right now
            VerseLength = self.__Get_Random_Length_Of_Verse()
            Sequence = StartingTokenOfThisVerse
            CurrentVerse = Sequence

            #Length is less than what is required by self.ngras
            if self.ngram > 2 and len(CurrentVerse.split()) == 1:
                NextWords = self.__ReverseBackOff(Sequence)
                CurrentVerse = NextWords
                Sequence = CurrentVerse

            #Now the sequence is ready for self.ngram probs
            for idxWord in range(len(CurrentVerse.split()),VerseLength):
                NextWords = self.__Get_Next_Word_From_Candidate(Sequence)
                Sequence = ' '.join(Sequence.split()[1:] + ([NextWords]))
                CurrentVerse += ' ' + NextWords
                
            CurrentVerse += '\n'
            if idxVerse != 0 and idxVerse % 2 == 1:
                CurrentVerse += '\n'
            Verse += CurrentVerse
        return Verse 

    def Print_Details(self):
        print('All ngrams',self.Vocabs.keys())
        for ngram in self.Vocabs.keys():
            print('ngram:',ngram)
            print('ngrams:',list(self.Vocabs[ngram].keys())[0:10])


In [11]:
ObjLM = Laplace_LM_Roman_Urdu(3)

AllTokens = np.load('../Data/All_Tokens_Roman_Urdu.npy')
AllTokens = [token.lower() for token in AllTokens]
StartingTokens = np.load('../Data/Starting_Tokens_Roman_Urdu.npy')
StartingTokens = [token.lower() for token in StartingTokens]
ObjLM.fit(AllTokens)

# ObjLM.Print_Details()

In [12]:
Pred = ObjLM.predict(StartingTokens)

In [13]:
print(Pred)

baghal mein ghair ki aaj aap sotay hain
taat o zuhad par tabiyat idhar

arsa جولاں tha sun-hwa wamandgy se
ik umar se aaya nah gaya aakhir

yaan yeh hijaab paas waza raah mein
ya koi gul khula asshk amde ke

zindagi mein maza nahi baqi zabt ka hosla
kya aakhir shab dil mein phir گریے ne

teri mohar o wafa ke baab aaye kar
hai yaaro saba se kuch to

niklana khuld se aadam ka suntay aaye
kon karta hai wafa ehad wafa aakhir shab

daam har mouj mein hai ke yeh
haazir ki kaayenaat hai kya jo kis


